#We read the posts.xml and Tags.xml files to extract the features that we feed to our regression model. The dictionar named "questions" contain all our features.

In [ ]:
# importing element tree
# under the alias of ET
import xml.etree.ElementTree as ET

# Passing the path of the
# xml document to enable the
# parsing process
tree = ET.parse('Posts.xml')

# getting the parent tag of
# the xml document
root = tree.getroot()


<Element 'posts' at 0x7f47a77a0590>
69


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print((root[0].attrib)['Tags'])
print((root[0].attrib)['CreationDate'])
print((root[0].attrib)['LastActivityDate'])
print((root[0].attrib)['AnswerCount'])

<ruby>
2014-09-29T20:06:18.613
2015-01-25T20:33:15.580
3


In [ ]:
from datetime import datetime
questions=dict()
for post in root:
  if post.attrib['PostTypeId']=="1":
    attributes=post.attrib
    postedTime= datetime.strptime(attributes['CreationDate'],'%Y-%m-%dT%H:%M:%S.%f')
    answerCount=attributes['AnswerCount']
    tags=attributes['Tags']
    if 'AcceptedAnswerId' in attributes:
      acceptedAnswerId=attributes['AcceptedAnswerId']
    else:
      acceptedAnswerId=""
    questions[post.attrib['Id']]={'Responses':[], 'Tags':tags, 'PostedTime':postedTime,'AcceptedAnswerId': acceptedAnswerId,'TitleLength': len(attributes['Title']),'AnswerCount':answerCount}

#questions

In [ ]:

responses=dict()
d=True
for post in root:
  if post.attrib['PostTypeId']=="2":
    attributes=post.attrib
    parentId=attributes['ParentId']
    replyTime= datetime.strptime(attributes['CreationDate'],'%Y-%m-%dT%H:%M:%S.%f')
    score=attributes['Score']
    
    postedTime=questions[parentId]['PostedTime']
    responseTime=(replyTime-postedTime).seconds/60
    if d==True:
      print(type(responseTime))
      d=False
    if 'OwnerUserId' in attributes:
      responses[post.attrib['Id']]={'OwnerId':attributes['OwnerUserId'], 'ParentId':parentId, 'ResponseTime':responseTime, 'Score':score}
      questions[parentId]['Responses'].append({'ResponderId':attributes['OwnerUserId'],'ResponseId':post.attrib['Id'], 'ResponseTime':responseTime, 'Score':score})
#responses

<class 'float'>


In [ ]:
questions

In [ ]:

for postId in questions:
  #print(postId)
  temp=questions[postId]['Tags'].split('>')[:-1]
  tagList=[]
  for s in temp:
    tagList.append(s[1:])

  questions[postId]['Tags']=tagList
  questions[postId]['Responses']=sorted(questions[postId]['Responses'],key=lambda x:x['ResponseTime'])
  #print(questions[postId]['Responses'])
  if len(questions[postId]['Responses'])>0:
    questions[postId]['EarliestResponseTime']=questions[postId]['Responses'][0]['ResponseTime']
  

#questions

In [ ]:
tree = ET.parse('Tags.xml')

# getting the parent tag of
# the xml document
tags = tree.getroot()
print(tags[0].attrib)
tagNames=[]
for tag in tags:
  tagNames.append(tag.attrib['TagName'])

tagNames

In [ ]:
questions

In [ ]:
from collections import defaultdict
def def1_value():
  return dict()


TagDict=defaultdict(def1_value) #[]

for postId in questions:
  tags=questions[postId]['Tags']
  for tag in tags:
    #TagDict[tag]=
    for response in questions[postId]['Responses']:
      if response['ResponderId'] in TagDict[tag].keys():
        TagDict[tag][response['ResponderId']].append(response['ResponseTime'])
      else:
        TagDict[tag][response['ResponderId']]=[response['ResponseTime']]

TagDict

In [ ]:
import numpy as np

avgRTdict={}
num_subs_t={} #[]
for tag in TagDict:
  for responderId in TagDict[tag]:
    rtList = TagDict[tag][responderId]
    Q1,Q3 = np.percentile(rtList , [25,75])
    IQR = Q3 - Q1
    ul = Q3+1.5*IQR
    ll = Q1-1.5*IQR
    rtListRemovingOutlier=[]
    for responseTime in rtList:
      if responseTime>=ll and responseTime<=ul:
        rtListRemovingOutlier.append(responseTime)
    avgResponseTime=sum(rtListRemovingOutlier) / len(rtListRemovingOutlier)
    if avgResponseTime<=60:      
      if tag not in avgRTdict:
        avgRTdict[tag]=[avgResponseTime]
      else:
        avgRTdict[tag].append(avgResponseTime)
  if tag not in avgRTdict:
    num_subs_t[tag]=0
  else:
    num_subs_t[tag]=len(avgRTdict[tag])


In [ ]:
def def1_value():
  return dict()
active_subscribers=defaultdict(def1_value)
for tag in TagDict:
  for response in TagDict[tag]:
    if len(TagDict[tag][response])>10:
      if response in active_subscribers[tag].keys():
          active_subscribers[tag][response].append(len(TagDict[tag][response]))
      else:
        active_subscribers[tag][response]=len(TagDict[tag][response])

In [ ]:
num_subs_ans={}
for tag in active_subscribers:
  if tag not in num_subs_ans:
    num_subs_ans[tag]=len(active_subscribers[tag])
num_subs_ans

In [ ]:
import pandas as pd

df = pd.read_csv('bodyLength.csv')

#print(df.head()) 


'\nx=df.loc[df["PostID"] ==-1]\ns=list(x["TagSpecificity"])\nif len(s)>0:\n  print(s[0])\n  print(type(s[0]))\nelse:\n  print("LOL")\n'

In [ ]:
from pickle import NONE
t=0
b=True
for postId in questions:
  q=int(postId)
  x=df.loc[df["PostID"] ==q]
  s=list(x["BodyLength"])
  if len(s)>0:
    questions[postId]['BodyLength']=s[0]
  else:
    questions[postId]['BodyLength']=NONE
    t+=1
    if b==True:
      print(postId)
      b=False

print(t)
#questions

0


In [ ]:
for id in questions:
  questions[id]['PostedTime']=str(questions[id]['PostedTime'])

In [ ]:
questions

In [ ]:
import pickle
def writeToPickle(data,location):
  with open(location, 'wb') as outfile:
    pickle.dump(data, outfile, protocol=pickle.HIGHEST_PROTOCOL)

writeToPickle(questions,'postInfo.pickle')

FileNotFoundError: ignored

In [ ]:
with open('postInfo.pickle', 'rb') as handle:
    questions = pickle.load(handle)
questions

In [ ]:
def def1_value():
  return dict()
active_subscribers=defaultdict(def1_value)
for tag in TagDict:
  for response in TagDict[tag]:
    if len(TagDict[tag][response])>10:
      if response in active_subscribers[tag].keys():
          active_subscribers[tag][response].append(len(TagDict[tag][response]))
      else:
        active_subscribers[tag][response]=len(TagDict[tag][response])

In [ ]:
#Tag information
# Passing the path of the
# xml document to enable the
# parsing process
tree = ET.parse('Tags.xml')

root = tree.getroot()

TagPopularityInfo={}

for tag in root:
  TagPopularityInfo[tag.attrib['TagName']]=int(tag.attrib['Count'])

TagPopularityInfo


#Next, we build the attributes dataframe from the dictionary questions containing all our model features proposed by Vasudev et al. 

In [ ]:
from pandas.compat.numpy.function import ALLANY_DEFAULTS
#Make feature set

df = pd.DataFrame(columns=['PostId','title_length','body_length','num_subs_ans','percent_subs_ans','num_subs_t','percent_subs_t','popular_tag','avg_popularity', 'response_time'])
#df= pd.DataFrame()
for question in questions:
  if 'EarliestResponseTime' not in questions[question]:
    continue
  
  allAttributes={}
  allAttributes['PostId']=question
  tag_list = questions[question]['Tags']
  total_num_subs_ans=0
  total_num_subs_t=0
  total_percent_num_subs_ans=0.0
  total_percent_num_subs_t = 0.0
  for tag in tag_list:
    if tag in num_subs_ans:
      total_num_subs_ans+=num_subs_ans[tag]
      total_percent_num_subs_ans += float(num_subs_ans[tag]) / len(TagDict[tag])
    if tag in num_subs_t:
      total_num_subs_t+=num_subs_t[tag]
      total_percent_num_subs_t += float(num_subs_t[tag]) / len(TagDict[tag])

  
  allAttributes['num_subs_ans'] = float(total_num_subs_ans)/ len(tag_list)
  allAttributes['percent_subs_ans'] = total_percent_num_subs_ans/ len(tag_list)
  allAttributes['num_subs_t'] = float(total_num_subs_t) / len(tag_list)
  allAttributes['percent_subs_t'] = total_percent_num_subs_t / len(tag_list)

  allAttributes['title_length'] = questions[question]['TitleLength']
  allAttributes['body_length'] = questions[question]['BodyLength']

  num_popular_tags=0

  for tag in tag_list:
    if tag in TagPopularityInfo:
      if TagPopularityInfo[tag] > 25:
        num_popular_tags+=1
  allAttributes['popular_tag'] = num_popular_tags
  questions[question]['NumPopularTags'] = num_popular_tags
  allAttributes['avg_popularity'] = questions[question]['AvgPopularity']
  allAttributes['response_time'] = questions[question]['EarliestResponseTime']
  if allAttributes['response_time']<=180:
    allAttributes['label'] = 1 
  else:
    allAttributes['label'] = 0
  #print(allAttributes)
  df = df.append(allAttributes, ignore_index=True)      


In [ ]:
df.to_csv('dataset.csv',index=False)

In [ ]:
import pickle

with open('tag_to_cluster.pickle', 'rb') as handle:
    tag_cluster = pickle.load(handle)

with open('cluster_to_avgPop.pickle', 'rb') as handle:
    cluster_avgPop = pickle.load(handle)

with open('clusters.pickle', 'rb') as handle:
    clusters = pickle.load(handle)

print(cluster_avgPop[tag_cluster['wordpress']])
clusters

from collections import defaultdict
def defineDict():
  return 0


302.875


'\ntags=[\'a\',\'b\', \'c\',\'d\', \'e\', \'f\']\ntag_cluster={\'a\':1,\'c\':1, \'b\':2, \'d\': 2, \'e\':2, \'f\': 3}\ncluster_avgPop={1:34, 2: 60, 3:5}\nclusters={1:[\'a\',\'c\',\'x\',\'y\',\'z\',\'g\',\'h\',\'i\',\'j\',\'k\'], 2: [\'b\',\'d\',\'e\',\'l\',\'m\',\'n\'], 3:[\'f\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\',\'t\']}\n#a,b,a,d,e,f #a->1:2\n#a,b,a,b,b,f # b->2: 3\n#\n#for one question\ndemo_dict=defaultdict(defineDict)\nstack=[]\nfor i in range(len(tags)):\n\n  if tags[i] not in stack:  \n\n    for j in range(len(tags)):\n      if tag_cluster[tags[i]]==tag_cluster[tags[j]]:\n        demo_dict[tag_cluster[tags[i]]]+=1\n        tags[j]=tags[i]\n        stack.append(tags[i])\n    \ntopic_relevance=0\nfor cluster in demo_dict:\n  print(str(cluster)+":")\n  x=cluster_avgPop[cluster]*(demo_dict[cluster]/len(clusters[cluster]))\n  print(x)\n  topic_relevance+=x\n\n\nprint(demo_dict)\nprint(topic_relevance)\n'

In [ ]:
import pandas as pd

df=pd.read_csv('dataset.csv', sep=',', header=0)
df

,PostId,title_length,body_length,num_subs_ans,percent_subs_ans,num_subs_t,percent_subs_t,popular_tag,avg_popularity,response_time,label
0,69,36,224,29.000000,0.076923,59.000000,0.156499,1,1065.0,4.300000,1.0
1,73,34,765,2.000000,0.031250,9.000000,0.140625,1,122.0,119.016667,1.0
2,84,30,145,4.000000,0.049383,15.000000,0.185185,1,250.0,725.383333,0.0
3,103,27,2087,21.000000,0.055851,43.000000,0.114362,1,1144.0,0.000000,1.0
4,106,23,42,25.000000,0.053419,57.000000,0.121795,1,1408.0,206.916667,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22201,82219,51,3637,18.333333,0.027509,36.666667,0.087923,2,925.0,648.433333,0.0
22202,82223,10,1555,39.000000,0.078453,54.500000,0.111563,2,2418.0,31.416667,1.0
22203,82233,30,521,0.000000,0.000000,11.000000,0.197306,2,65.0,173.483333,1.0
22204,82236,77,359,12.000000,0.047619,32.000000,0.126984,1,640.0,22.716667,1.0


In [ ]:
PostId=list(df['PostId'])
#PostId

id_tags=dict()
for id in PostId:
  if str(id) in questions:
    id_tags[id]=questions[str(id)]['Tags']

id_tags

In [ ]:
post_topicRelevance=dict()
p=list(cluster_avgPop.values())
defaultVal=min(p)/10.0

for id in id_tags:
  demo_dict=defaultdict(defineDict)
  stack=[]
  tags=id_tags[id]
  for i in range(len(tags)):

    if tags[i] not in stack and tags[i] in tag_cluster:  
      for j in range(len(tags)):
        if tags[j] in tag_cluster:
          if tag_cluster[tags[i]]==tag_cluster[tags[j]]:
              demo_dict[tag_cluster[tags[i]]]+=1
              tags[j]=tags[i]
              stack.append(tags[i])
      
  topic_relevance=0
  if len(demo_dict)>0:
    for cluster in demo_dict:
      print(str(cluster)+":")
      x=cluster_avgPop[cluster]*(demo_dict[cluster]/len(clusters[cluster]))
      print(x)
      topic_relevance+=x
  else:
    topic_relevance=defaultVal
  post_topicRelevance[id]=topic_relevance



In [ ]:
topicRelevance=list()
for id in PostId:
  topicRelevance.append(post_topicRelevance[id])

df['topic_relevance']=topicRelevance

time=list(df['response_time'])
label=list(df['label'])

del df['response_time']
del df['label']


df['response_time']=time
df['label']=label
df

,PostId,title_length,body_length,num_subs_ans,percent_subs_ans,num_subs_t,percent_subs_t,popular_tag,avg_popularity,topic_relevance,response_time,label
0,69,36,224,29.000000,0.076923,59.000000,0.156499,1,1065.0,16.287500,4.300000,1.0
1,73,34,765,2.000000,0.031250,9.000000,0.140625,1,122.0,17.400000,119.016667,1.0
2,84,30,145,4.000000,0.049383,15.000000,0.185185,1,250.0,8.244444,725.383333,0.0
3,103,27,2087,21.000000,0.055851,43.000000,0.114362,1,1144.0,14.553571,0.000000,1.0
4,106,23,42,25.000000,0.053419,57.000000,0.121795,1,1408.0,43.267857,206.916667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22201,82219,51,3637,18.333333,0.027509,36.666667,0.087923,2,925.0,264.481481,648.433333,0.0
22202,82223,10,1555,39.000000,0.078453,54.500000,0.111563,2,2418.0,690.857143,31.416667,1.0
22203,82233,30,521,0.000000,0.000000,11.000000,0.197306,2,65.0,32.748889,173.483333,1.0
22204,82236,77,359,12.000000,0.047619,32.000000,0.126984,1,640.0,30.260000,22.716667,1.0


In [ ]:
df.to_csv('datasetRelevance.csv',index=False)

#Scaling

In [ ]:
data=pd.read_csv('datasetRelevance.csv')
data

,PostId,title_length,body_length,num_subs_ans,percent_subs_ans,num_subs_t,percent_subs_t,popular_tag,avg_popularity,topic_relevance,response_time,label
0,69,36,224,29.000000,0.076923,59.000000,0.156499,1,1065.0,16.287500,4.300000,1.0
1,73,34,765,2.000000,0.031250,9.000000,0.140625,1,122.0,17.400000,119.016667,1.0
2,84,30,145,4.000000,0.049383,15.000000,0.185185,1,250.0,8.244444,725.383333,0.0
3,103,27,2087,21.000000,0.055851,43.000000,0.114362,1,1144.0,14.553571,0.000000,1.0
4,106,23,42,25.000000,0.053419,57.000000,0.121795,1,1408.0,43.267857,206.916667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22201,82219,51,3637,18.333333,0.027509,36.666667,0.087923,2,925.0,264.481481,648.433333,0.0
22202,82223,10,1555,39.000000,0.078453,54.500000,0.111563,2,2418.0,690.857143,31.416667,1.0
22203,82233,30,521,0.000000,0.000000,11.000000,0.197306,2,65.0,32.748889,173.483333,1.0
22204,82236,77,359,12.000000,0.047619,32.000000,0.126984,1,640.0,30.260000,22.716667,1.0


In [ ]:
def min_max_scaling(df):
    # copy the dataframe
    features=['PostId','response_time','label']
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
      if column not in features:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm
    
# call the min_max_scaling function
df_normalized = min_max_scaling(data)
df_normalized.to_csv('datasetRelevanceNorm.csv',index=False)

In [ ]:
def z_score(df):
    # copy the dataframe
    df_std = df.copy()
    features=['PostId','response_time','label']
    # apply the z-score method
    for column in df_std.columns:
      if column not in features:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
        
    return df_std
    
# call the z_score function
df_standardized = z_score(data)

df_standardized.to_csv('datasetRelevanceStandard.csv',index=False)

#Regression Model

In [ ]:
from mlxtend.preprocessing import minmax_scaling


In [ ]:
# Load Dataset from CSV
import pandas as pd
import numpy as np

data = pd.read_csv('datasetRelevanceNorm.csv', sep=',', header=0)
data = data.loc[data['response_time'] < 180]


In [ ]:
data

,PostId,title_length,body_length,num_subs_ans,percent_subs_ans,num_subs_t,percent_subs_t,popular_tag,avg_popularity,topic_relevance,response_time,label
0,69,0.198582,0.005868,0.508772,0.230769,0.694118,0.156499,0.2,0.291347,0.013171,4.300000,1.0
1,73,0.184397,0.022067,0.035088,0.093750,0.105882,0.140625,0.2,0.033133,0.014081,119.016667,1.0
3,103,0.134752,0.061649,0.368421,0.167553,0.505882,0.114362,0.2,0.312979,0.011753,0.000000,1.0
5,107,0.219858,0.002605,1.000000,0.236515,0.917647,0.107884,0.2,1.000000,0.282346,15.316667,1.0
8,147,0.290780,0.048504,0.000000,0.000000,0.017647,0.750000,0.0,0.000821,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22199,82209,0.078014,0.044163,0.333333,0.078838,0.317647,0.075961,0.4,0.337897,0.296495,66.133333,1.0
22202,82223,0.014184,0.045720,0.684211,0.235358,0.641176,0.111563,0.4,0.661829,0.564842,31.416667,1.0
22203,82233,0.156028,0.014761,0.000000,0.000000,0.129412,0.197306,0.4,0.017525,0.026633,173.483333,1.0
22204,82236,0.489362,0.009910,0.210526,0.142857,0.376471,0.126984,0.2,0.174973,0.024598,22.716667,1.0


In [ ]:
data['response_time'] = data['response_time'].div(60)

In [ ]:
#data['response_time'] = data['response_time'].div(60).round(2)
y = np.array(data.iloc[:,10])
x= np.array(data.iloc[:,1:10])

In [ ]:
x.shape

(10806, 8)

In [ ]:
data

,PostId,title_length,body_length,num_subs_ans,percent_subs_ans,num_subs_t,percent_subs_t,popular_tag,avg_popularity,topic_relevance,response_time,label
0,69,0.198582,0.005868,0.508772,0.230769,0.694118,0.156499,0.2,0.291347,0.013171,0.071667,1.0
1,73,0.184397,0.022067,0.035088,0.093750,0.105882,0.140625,0.2,0.033133,0.014081,1.983611,1.0
3,103,0.134752,0.061649,0.368421,0.167553,0.505882,0.114362,0.2,0.312979,0.011753,0.000000,1.0
5,107,0.219858,0.002605,1.000000,0.236515,0.917647,0.107884,0.2,1.000000,0.282346,0.255278,1.0
8,147,0.290780,0.048504,0.000000,0.000000,0.017647,0.750000,0.0,0.000821,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22199,82209,0.078014,0.044163,0.333333,0.078838,0.317647,0.075961,0.4,0.337897,0.296495,1.102222,1.0
22202,82223,0.014184,0.045720,0.684211,0.235358,0.641176,0.111563,0.4,0.661829,0.564842,0.523611,1.0
22203,82233,0.156028,0.014761,0.000000,0.000000,0.129412,0.197306,0.4,0.017525,0.026633,2.891389,1.0
22204,82236,0.489362,0.009910,0.210526,0.142857,0.376471,0.126984,0.2,0.174973,0.024598,0.378611,1.0


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
type(x_train)

numpy.ndarray

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train, y_train)

In [ ]:
y_pred = reg.predict(x_test)
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.67973976724152
Mean Squared Error: 0.6549448544777341
Root Mean Squared Error: 0.8092866330773876


In [ ]:
reg.score(x_test, y_test)

0.041051346944080636

Mean Absolute Error: 5.492905125155626
Mean Squared Error: 44.456624053039995
Root Mean Squared Error: 6.66758007473776

#MLP Regressor

In [ ]:
#MLP Regressor
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(hidden_layer_sizes= (200,), max_iter=500, early_stopping=True, validation_fraction=0.2).fit(x_train, y_train)

In [ ]:
y_pred = regr.predict(x_test)
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.6545049308551926
Mean Squared Error: 0.6139884814022322
Root Mean Squared Error: 0.7835741709641992


In [ ]:
regr.score(x_test,y_test)

0.03724700197674202

#K-fold Cross Validation for MLP Regressor

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor

activations= ['relu', 'tanh', ]
hidden_layers = [(100,), (150,), (200,), (250,), (300,)]
max_iters= [200, 300, 400]
average_scores = []
for activation_function in activations:
  for hidden_layer in hidden_layers:
    for max_it in max_iters:
      info={}
      info['hidden_layer_size'] = str(hidden_layer)
      info['activation'] = activation_function
      info['max_iteration'] = str(max_it)
      regr = MLPRegressor(hidden_layer_sizes= hidden_layer, activation= activation_function, max_iter=max_it, early_stopping=True, validation_fraction=0.2)
      scores = cross_val_score(regr, x_train, y_train, cv=10)
      
      info['average_error'] = str(np.mean(scores, dtype=np.float64))
      average_scores.append(info)
      print(np.mean(scores, dtype=np.float64))


0.037804958475994896
0.03650947914567826
0.036946910877220206
0.03794827405960362
0.03760094768460308
0.03699177552586334
0.03757528654783267
0.03673493697617754
0.0369095651333568
0.037748838855286196
0.03650462836857972
0.03650878230595167
0.03699474269331889
0.03806586439913264
0.03729360669936675
0.03526444165958712
0.03564588704512445
0.035469186451493905
0.03611040618676362
0.036241618492684605
0.035313040986084364
0.035371320651165085
0.03541282300817354
0.03491809371603759
0.035009877306818295
0.036001725035037645
0.035215481022438266
0.03499655502343635
0.03515811618276445
0.03473178278709514


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor


early_stoppings= [ True, False]
valid_fractions= [0.1, 0.15, 0.20]
average_scores = []

for early in early_stoppings:
  if early:
    for valid in valid_fractions:
      info={}
      info['valid_fraction'] = valid
      regr = MLPRegressor(hidden_layer_sizes= (250,), activation= 'relu', max_iter= 300, early_stopping=early, validation_fraction=valid)
      scores = cross_val_score(regr, x_train, y_train, cv=10)
      info['average_error'] = str(np.mean(scores, dtype=np.float64))
      average_scores.append(info)
      print(np.mean(scores, dtype=np.float64))
  else:
    regr = MLPRegressor(hidden_layer_sizes= (250,), activation= 'relu', max_iter= 300, early_stopping=False)
    scores = cross_val_score(regr, x_train, y_train, cv=10)
    info['average_error'] = str(np.mean(scores, dtype=np.float64))
    average_scores.append(info)
    print(np.mean(scores, dtype=np.float64))


0.035016284910508136
0.03858746808460099
0.03904952985504612
0.0385337924028137


In [ ]:
average_scores

[{'average_error': '0.035016284910508136', 'valid_fraction': 0.1},
 {'average_error': '0.03858746808460099', 'valid_fraction': 0.15},
 {'average_error': '0.0385337924028137', 'valid_fraction': 0.2},
 {'average_error': '0.0385337924028137', 'valid_fraction': 0.2}]

In [ ]:
average_scores

#Linear Regression from scratch

In [ ]:
#code source: https://machinelearningmastery.com/implement-simple-linear-regression-scratch-python/
def compute_cost(X, y, theta):
  m=X.shape[0]

  predictions = X.dot(theta)
  #print('predictions= ', predictions[:5])
  errors = np.subtract(predictions, y)
  #print('errors= ', errors[:5]) 
  sqrErrors = np.square(errors)
  J = 1/(2 * m) * errors.T.dot(errors)

  return J

In [ ]:
def gradient_descent(X, y, theta, alpha, iterations):
 
  cost_history = np.zeros(iterations)
  m=X.shape[0]
  for i in range(iterations):
    predictions = X.dot(theta)
    #print('predictions= ', predictions[:5])
    errors = np.subtract(predictions, y)
    #print('errors= ', errors[:5])
    sum_delta = (alpha / m) * X.transpose().dot(errors);
    #print('sum_delta= ', sum_delta[:5])
    theta = theta - sum_delta;

    cost_history[i] = compute_cost(X, y, theta)  

  return theta, cost_history

In [ ]:
theta = np.zeros(x_train.shape[1])
iterations = 400;
alpha = 0.15;

In [ ]:
theta.shape

(8,)

In [ ]:
theta, cost_history = gradient_descent(x_train, y_train, theta, alpha, iterations)


In [ ]:
y_pred = x_test.dot(theta)


In [ ]:
y_pred

array([1.18645806, 0.94363006, 1.44414382, ..., 1.28438139, 0.6517122 ,
       1.06270608])

In [ ]:
y_test

array([0.32333333, 0.09416667, 1.02527778, ..., 2.57138889, 1.23833333,
       0.21722222])

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.7140450023601941
Mean Squared Error: 0.7591796498695186
Root Mean Squared Error: 0.8713091586053245


#K-fold Cross validation Linear Regression

In [ ]:
# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold
from sklearn import metrics

# data sample
# prepare cross validation
learning_rates= [0.01, 0.02, 0.05, 0.75, 0.95, 1.0, 1.2, 1.5]
iter = [200, 300, 400, 500]
allinfo = []
for alpha in learning_rates:
  for it in iter:
    score = []
    m = 1   
    info = {}
    info['learning_rate'] = str(alpha)
    info['max_iter'] = str(it)
    kfold = KFold(n_splits=10, shuffle=True, random_state=1)
    # enumerate splits
    for train, test in kfold.split(x_train):
      #print('train: %s, test: %s' % (X[train], X[test]))
      X_train= x_train[train]
      X_test= x_train[test]
      Y_train = y_train[train]
      Y_test = y_train[test]
      theta = np.zeros(X_train.shape[1])

      theta, cost_history = gradient_descent(X_train, Y_train, theta, alpha, it)
      Y_pred = X_test.dot(theta)
      score.append(np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)) - 0.09)
    
    avg_score = sum(score) / len(score)
    print(avg_score)
    info['average_rmse'] = avg_score
    allinfo.append(info)


In [ ]:
allinfo